# Use Canny edge detector to reduce search area - Zuohe Zheng

In [ ]:
import numpy as np
import scipy as sp
from scipy import io
import cv2 as cv 
import matplotlib.pyplot as plt
import pylab
import os # for reading all files in a folder
pylab.rcParams['figure.figsize'] = (12.0, 10.0)

In [ ]:
# Canny Edge detector
images = []
iFrame = 0
folder = 'Pattern01/'

for frameNum in sorted(os.listdir(folder)):
        images.append(cv.imread(folder+frameNum))
        iFrame += 1
        
imgHeight, imgWidth, colors = images[0].shape
edges0 = cv.Canny(images[0],0,100)
plt.imshow(edges0)

# Use the four edges of the black square to set a boundary
up_bound = np.zeros(iFrame)
low_bound = np.zeros(iFrame)
left_bound = np.zeros(iFrame)
right_bound = np.zeros(iFrame)

for iframe in range(iFrame):
    img = images[iframe]
    edges = cv.Canny(img,0,100)
    up_y = np.where(edges[0:300,300]>0)
    low_y = np.where(edges[301:500,300]>0)
    left_x = np.where(edges[300,0:300]>0)
    right_x = np.where(edges[300,301:600]>0)
    
    up_b = np.int(np.max(up_y)-50)
    low_b = np.int(np.min(low_y)+300+50)
    left_b = np.int(np.max(left_x)-50)
    right_b = np.int(np.min(right_x)+300+50)
    
    # constrain the boundary in the image area
    up_bound[iframe] = np.max([up_b,0])
    low_bound[iframe] = np.min([low_b,imgHeight])
    left_bound[iframe] = np.max([left_b,0])
    right_bound[iframe] = np.min([right_b,imgWidth])

In [ ]:
test_ur = HW2_Practical9c('ur',2000,15,up_bound,low_bound,left_bound,right_bound)
test_ul = HW2_Practical9c('ul',2000,15,up_bound,low_bound,left_bound,right_bound)
test_ll = HW2_Practical9c('ll',3000,15,up_bound,low_bound,left_bound,right_bound)
test_lr = HW2_Practical9c('lr',2000,15,up_bound,low_bound,left_bound,right_bound)

In [ ]:
# Load all images in folder
images = []
nFrame = 0
folder = 'Pattern01/'
for frameNum in sorted(os.listdir(folder)):
    images.append(cv.imread(folder+frameNum))
    nFrame += 1
# # plot first image 
# plt.imshow(images[0])
# plt.show()


# Coordinates of the known target object (a dark square on a plane) in 3D:
XCart = np.array([[-50, -50,  50,  50],
          [50, -50, -50,  50],
            [0, 0, 0, 0]])

# These are some approximate intrinsics for this footage.
K = np.array([[640, 0, 320],
          [0, 512, 256],
            [0, 0, 1]])

# Define 3D points of wireframe object.
XWireFrameCart = np.array([[-50, -50,  50,  50, -50, -50,  50,  50],
          [50, -50, -50,  50, 50, -50, -50,  50],
            [0, 0, 0, 0, -100, -100, -100, -100, ]])

In [ ]:
# ================================================
for iFrame in range(nFrame):
    print('Processing Frame', iFrame)
    xImCart = np.array([test_ll[iFrame,:].T, test_ul[iFrame,:].T, test_ur[iFrame,:].T, test_lr[iFrame,:].T]).T

    # get a frame from footage 
    im = images[iFrame]

    # Draw image and 2d points
    plt.imshow(im)
    plt.scatter(x = xImCart[0,:], y = xImCart[1,:],c = 'r')
#     plt.show()


    #TO DO: Use your routine to calculate TEst the extrinsic matrix relating the
    #plane position to the camera position.
    #T = estimatePlanePose(xImCart, XCart, K);
    TEst = estimatePlanePose(xImCart,XCart,K)
    
    # TO DO: Draw a wire frame cube using data XWireFrameCart. You need to
    # 1) project the vertices of a 3D cube through the projective camera;
    # 2) draw lines betweeen the resulting 2d image points.
    # Note: CONDUCT YOUR CODE FOR DRAWING XWireFrameCart HERE
    
    XWireFrameCartProjected = projectiveCamera(K,TEst,XWireFrameCart)
    plt.plot(XWireFrameCartProjected[0,],XWireFrameCartProjected[1,],'g.')

    # Connect the points to draw the frame of the cube
    for cPoint in range(4):
        plt.plot([XWireFrameCartProjected[0,cPoint],XWireFrameCartProjected[0,cPoint+4]], \
                 [XWireFrameCartProjected[1,cPoint],XWireFrameCartProjected[1,cPoint+4]],'g-') 
    for cPoint in range(3):
        plt.plot([XWireFrameCartProjected[0,cPoint],XWireFrameCartProjected[0,cPoint+1]], \
                 [XWireFrameCartProjected[1,cPoint],XWireFrameCartProjected[1,cPoint+1]],'g-') 
        plt.plot([XWireFrameCartProjected[0,cPoint+4],XWireFrameCartProjected[0,cPoint+5]], \
                 [XWireFrameCartProjected[1,cPoint+4],XWireFrameCartProjected[1,cPoint+5]],'g-')
    plt.plot([XWireFrameCartProjected[0,0],XWireFrameCartProjected[0,3]], \
                 [XWireFrameCartProjected[1,0],XWireFrameCartProjected[1,3]],'g-') 
    plt.plot([XWireFrameCartProjected[0,4],XWireFrameCartProjected[0,7]], \
                 [XWireFrameCartProjected[1,4],XWireFrameCartProjected[1,7]],'g-') 

    plt.axis('off')
    plt.show()


The new function of HW2_Practical9c was changed to restrict the position of particles by four boundaries instead of the shape of the image, to reduce the search area. As the seach area shrinks, the noise level can be smaller to concentrate particles. The result shows that now particles can land around only the edges of the black square, which reduce the possibility of error tracking. 

In [ ]:
def computeLikelihood(image, template):
    methods = [cv.TM_CCOEFF, cv.TM_CCOEFF_NORMED, cv.TM_CCORR,
            cv.TM_CCORR_NORMED, cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    likelihood = cv.matchTemplate(image, template, methods[0])  
    pad_first = int(template.shape[0])
    pad_second = int(template.shape[1])
    pad_amounts = ((0, pad_first-1), (0, pad_second-1))
    likelihood = np.pad(likelihood, pad_amounts, 'constant')
    likelihood[likelihood<0] = 0 # to avoid negative weights 
    kernel = np.ones((10,10),np.float32)/100
    smoothed = cv.filter2D(likelihood,-1,kernel) 
    return smoothed 

In [ ]:
def HW2_Practical9c(corner,particlesNum,std,up_bound,low_bound,left_bound,right_bound):
    template = sp.io.loadmat(corner+'.mat')['pixelsTemplate']
    # let's show the template
#     print('We are matching this template with shape: ', template.shape)
#     plt.imshow(template)
#     plt.show()

    # Load all images in folder
    images = []
    iFrame = 0
    folder = 'Pattern01/'
    for frameNum in sorted(os.listdir(folder)):
        images.append(cv.imread(folder+frameNum))
        iFrame += 1
#     # plot first image 
#     plt.imshow(images[0])
#     plt.show()

    imgHeight, imgWidth, colors = images[0].shape
    numParticles = particlesNum;
    weight_of_samples = np.ones((numParticles,1))

    # TO DO: normalize the weights (may be trivial this time) [done]
    weight_of_samples = weight_of_samples / np.sum(weight_of_samples) #replace this 

    # Initialize which samples from "last time" we want to propagate: all of
    # them!:
    samples_to_propagate = range(0, numParticles)


    # ============================
    # NOT A TO DO: You don't need to change the code below, but eventually you may
    # want to vary the number of Dims (compare for example to lab 9b) 
    numDims_w = 2;
    # Here we randomly initialize some particles throughout the space of w:
    particles_old = np.random.rand(numParticles, numDims_w)
    particles_old[:,0] = particles_old[:,0] * imgHeight
    particles_old[:,1] = particles_old[:,1] * imgWidth
    # ============================

    #Initialize a temporary array r to store the per-frame MAP estimate of w. This is what we'll return in the end.
    r = np.zeros((iFrame, numDims_w));

    for iTime in range(iFrame):
#         print('Processing Frame', iTime)
        # TO DO: compute the cumulative sume of the weights. [done]
#         cum_hist_of_weights = np.linspace(0, 1, numParticles) # replace this
        cum_hist_of_weights = np.zeros(numParticles)
        for c in range(numParticles):
            cum_hist_of_weights[c] = np.sum(weight_of_samples[:c+1])
        #print(weight_of_samples)


        # ==============================================================
        # Resample the old distribution at time t-1, and select samples, favoring
        # those that had a higher posterior probability.
        # ==============================================================
        samples_to_propagate = np.zeros(numParticles, dtype=np.int32)

        # Pick random thresholds in the cumulative probability's range [0,1]:
        some_threshes = np.random.rand(numParticles)


        # For each random threshold, find which sample in the ordered set is
        # the first one to push the cumulative probability above that
        # threshold, e.g. if the cumulative histogram goes from 0.23 to 0.26
        # between the 17th and 18th samples in the old distribution, and the
        # threshold is 0.234, then we'll want to propagate the 18th sample's w
        # (i.e. particle #18).

        for sampNum in range(numParticles): 
            thresh = some_threshes[sampNum]
            for index in range (numParticles):
                if cum_hist_of_weights[index] > thresh:
                    break
            samples_to_propagate[sampNum] = index

        # Note: it's ok if some of the old particles get picked repeatedly, while
        # others don't get picked at all.


        # =================================================
        # Visualize if you want
        # =================================================
#         plt.title('Cumulative histogram of probabilities for sorted list of particles')
#         plt.plot(np.zeros(numParticles), some_threshes,'b.')
#         plt.plot(range(0, numParticles), cum_hist_of_weights, 'rx-')
#         which_sample_ids = np.unique(samples_to_propagate)
#         how_many_of_each = np.bincount(np.ravel(samples_to_propagate))
#         for k in range(len(which_sample_ids)):
#            plt.plot(which_sample_ids[k], 0, 'bo-', markersize = 3 * how_many_of_each[k], markerfacecolor='white')
#         plt.xlabel('Indeces of all available samples, with larger blue circles for frequently re-sampled particles\n(Iteration %01d)' % iTime)
#         plt.ylabel('Cumulative probability');
#         plt.show()
        # =================================================
        # =================================================

        # Predict where the particles we sampled from the old distribution of 
        # state-space will go in the next time-step. This means we have to apply 
        # the motion model to each old sample.
        particles_new = np.zeros_like(particles_old)
        for particleNum in range(numParticles):
            # TO DO: Incorporate some noise, e.g. Gaussian noise with std 20,
            # into the current location (particles_old), to give a Brownian
            # motion model.
            particles_new[particleNum, :] = particles_old[samples_to_propagate[particleNum], :] + np.random.normal(0,std,numDims_w)
            #replace this
            
        # TO DO: Not initially, but change the motion model above to have
        # different degrees of freedom, and optionally completely different
        # motion models. See Extra Credit for more instructions.

        #calculate likelihood function
        likelihood = computeLikelihood(images[iTime], template)

# #         plot results
#         f, axarr = plt.subplots(1, 2)
#         axarr[0].imshow(images[iTime])
#         axarr[0].set_title('Particles')
#         # now draw the particles onto the image
#         axarr[0].plot(particles_new[:,1]+template.shape[1]/2, particles_new[:,0]+template.shape[0]/2, 'rx')

#         #plot the likelihood
#         axarr[1].imshow(likelihood)
#         axarr[1].set_title('Likelihood')

        # From here we incorporate the data for the new state (time t):
        # The new particles accompanying predicted locations in state-space
        # for time t, are missing their weights: how well does each particle
        # explain the observations x_t?
        for particleNum in range(numParticles):

            # Convert the particle from state-space w to measurement-space x:
            # Note: that step is trivial here since both are in 2D space of image
            # coordinates

            # Within the loop, we evaluate the likelihood of each particle:
            particle = particles_new[particleNum, :]
            # Check that the predicted location is a place we can really evaluate
            # the likelihood.
            inFrame = particle[0] >= up_bound[iTime] and  particle[0] <= low_bound[iTime] and particle[1] >= left_bound[iTime] and particle[1] <= right_bound[iTime]            
            if inFrame:
                minX = particle[1]
                minY = particle[0]

                weight_of_samples[particleNum] = likelihood[int(minY), int(minX)]

            else:
                weight_of_samples[particleNum] = 0.0

        # TO DO: normalize the weights [done]
        weight_of_samples = weight_of_samples / np.sum(weight_of_samples) # replace this
        
        # find the location of the particle with highest weight
        indices = np.argsort(weight_of_samples,axis = 0)
        bestScoringParticles = particles_new[np.squeeze(indices[-15:]), :]
#         plt.plot(bestScoringParticles[:,1], bestScoringParticles[:,0], 'rx')
        # Return the MAP of middle position. Add template.shape/2 because matchTemplate finds the position of the upper left corner 
        # of the template. We want to plot the centre of the template. 
        r[iTime,:] = bestScoringParticles[-1,1]+template.shape[1]/2,bestScoringParticles[-1,0]+template.shape[0]/2
#         print(r[iTime,:])   
#         plt.show()

#         #print the original image and the position of the tracked corner.
#         plt.imshow(images[iTime])
#         plt.plot(r[iTime,0],r[iTime,1],'rx')
#         plt.show()
        # Now we're done updating the state for time t. 
        # For Condensation, just clean up and prepare for the next round of 
        # predictions and measurements:
        particles_old = particles_new

    return r

In [ ]:
#The goal of this function is to project points in XCart through projective camera
#defined by intrinsic matrix K and extrinsic matrix T.
def projectiveCamera(K,T,XCart):
    
    # TO DO: Replace this
    # XImCart =

    # TO DO: Convert Cartesian 3d points XCart to homogeneous coordinates XHom
    # by appending a row of 1 to the original point coordinates to form [u,v,w,1]
    XHom = np.concatenate((XCart, np.ones((1,XCart.shape[1]))), axis=0)
    # TO DO: Apply extrinsic matrix to XHom, to move to frame of reference of camera
    # = lambda * [x',y',1, 1/lambda]
    xCamHom1 = T @ XHom
    # TO DO: Project points into normalized camera coordinates xCamHom (remove 4th row)
    # = lambda * [x',y',1]
    xCamHom = xCamHom1[0:3,:]
    # TO DO: Move points to image coordinates xImHom by applying intrinsic matrix
    # = lambda * [x,y,1]
    xImHom = K @ xCamHom
    # TO DO: Convert points back to Cartesian coordinates xImCart
    # x = (lambda * x) / lambda
    # y = (lambda * y) / lambda
    XImCart = xImHom[0:2,:] / np.tile([xImHom[2,:]],(2,1))
    return XImCart

In [ ]:
def solveAXEqualsZero(A):
    # TO DO: Write this routine - it should solve Ah = 0   
    h = np.zeros(shape = [np.size(A),1])
    [U,L,Vt] = np.linalg.svd(A)
    V = np.transpose(Vt)
    h = V[:,-1]
    return h

In [ ]:
# Goal of function is to estimate pose of plane relative to camera (extrinsic matrix)
# given points in image xImCart, points in world XCart and intrinsic matrix K

def estimatePlanePose(XImCart,XCart,K):

    # TO DO: replace this
    #T = 

    # TO DO: Convert Cartesian image points XImCart to homogeneous representation XImHom
    # by appending a row of 1 to the original point coordinates to form [x,y,1]
    XImHom = np.concatenate((XImCart, np.ones((1,XImCart.shape[1]))), axis=0)
    
    # TO DO: Convert image co-ordinates XImHom to normalized camera coordinates XCamHom    
    # multiply with inverse of intrinsic matrix
    XCamHom = np.linalg.inv(K) @ XImHom
    
    # TO DO: Estimate homography H mapping homogeneous (x,y) coordinates of positions
    # in real world to XCamHom (convert XCamHom to Cartesian, calculate the homography) -
    # use the routine you wrote for Practical 1B
    
    # Extract the u and v coordinates of Cartesian 3d points XCart
    XCart = XCart[0:2,:]
    
    # Convert XCart to homogeneous coordinates XCartHom
    # [u,v] to [u,v,1]
    XCartHom = np.concatenate((XCart, np.ones((1,XCart.shape[1]))), axis=0)
    
    # Then construct the matrix A, size (n_points,9) 
    n_points = np.shape(XCamHom)[1]
    A = np.zeros(shape = [n_points*2,9])
    for n in range(n_points):
        A[2*n,[0,1,2]] = 0
        A[2*n,[3,4,5]] = -XCartHom[:,n]
        A[2*n,[6,7,8]] = XCartHom[:,n] * XCamHom[1,n]
        A[2*n+1,[0,1,2]] = XCartHom[:,n]
        A[2*n+1,[3,4,5]] = 0
        A[2*n+1,[6,7,8]] = -XCartHom[:,n] * XCamHom[0,n]
    
    # Solve Ah = 0
    h = solveAXEqualsZero(A)
    # Reshape h into the matrix H, values of h go first into rows of H
    h_width = np.sqrt(np.size(h))
    h_width = int(h_width)
    H = np.reshape(h,[h_width,h_width])
     
    # TO DO: Estimate first two columns of rotation matrix R from the first two
    # columns of H using the SVD
    # SVD decomposition of the first two columns of H
    [U,L,Vt] = np.linalg.svd(H[:,0:2])
    R = np.zeros(shape=[h_width,h_width])
    # Replace L with [1,0;0,1;0,0] to form the first two columns of R
    l = np.array([[1,0],[0,1],[0,0]])
    R[:,0:2] = U @ l @ Vt

    # TO DO: Estimate the third column of the rotation matrix by taking the cross
    # product of the first two columns
    R[:,2] = np.cross(R[:,0],R[:,1])
        
    # TO DO: Check that the determinant of the rotation matrix is positive - if
    # not then multiply last column by -1.
    if np.linalg.det(R) <= 0 :
        R[:,-1] = - R[:,-1]
    
    # TO DO: Estimate the translation t by finding the appropriate scaling factor k
    # and applying it to the third colulmn of H
    # Find translation scaling factor between old and new values
    Lamb = H / R
    lamb = np.sum(Lamb[0:3,0:2]) / 6
    t = H[:,-1] / lamb
    
    # TO DO: Check whether t_z is negative - if it is then multiply t by -1 and
    # the first two columns of R by -1.
    if t[-1] < 0 :
        t = -t
        R[:,0:2] = -R[:,0:2]
    
            
    # TO DO: Assemble transformation into matrix form
    # append [Tx,Ty,Tz] to R as the last column
    # then append [0,0,0,1] as the last row to form the final transformation matrix
    t = np.reshape(t,[3,1])
    T = np.concatenate((R, t), axis=1)
    T = np.concatenate((T, np.array([[0,0,0,1]])), axis=0)
    
    return T 

In [ ]:
def HW2_Practical9c_show(corner,particlesNum,std,up_bound,low_bound,left_bound,right_bound):
    template = sp.io.loadmat(corner+'.mat')['pixelsTemplate']
    # let's show the template
    print('We are matching this template with shape: ', template.shape)
    plt.imshow(template)
    plt.show()

    # Load all images in folder
    images = []
    iFrame = 0
    folder = 'Pattern01/'
    for frameNum in sorted(os.listdir(folder)):
        images.append(cv.imread(folder+frameNum))
        iFrame += 1
    # plot first image 
    plt.imshow(images[0])
    plt.show()

    imgHeight, imgWidth, colors = images[0].shape
    numParticles = particlesNum;
    weight_of_samples = np.ones((numParticles,1))

    # TO DO: normalize the weights (may be trivial this time) [done]
    weight_of_samples = weight_of_samples / np.sum(weight_of_samples) #replace this 

    # Initialize which samples from "last time" we want to propagate: all of
    # them!:
    samples_to_propagate = range(0, numParticles)


    # ============================
    # NOT A TO DO: You don't need to change the code below, but eventually you may
    # want to vary the number of Dims (compare for example to lab 9b) 
    numDims_w = 2;
    # Here we randomly initialize some particles throughout the space of w:
    particles_old = np.random.rand(numParticles, numDims_w)
    particles_old[:,0] = particles_old[:,0] * imgHeight
    particles_old[:,1] = particles_old[:,1] * imgWidth
    # ============================

    #Initialize a temporary array r to store the per-frame MAP estimate of w. This is what we'll return in the end.
    r = np.zeros((iFrame, numDims_w));

    for iTime in range(0,21):
        print('Processing Frame', iTime)
        # TO DO: compute the cumulative sume of the weights. [done]
#         cum_hist_of_weights = np.linspace(0, 1, numParticles) # replace this
        cum_hist_of_weights = np.zeros(numParticles)
        for c in range(numParticles):
            cum_hist_of_weights[c] = np.sum(weight_of_samples[:c+1])
        #print(weight_of_samples)


        # ==============================================================
        # Resample the old distribution at time t-1, and select samples, favoring
        # those that had a higher posterior probability.
        # ==============================================================
        samples_to_propagate = np.zeros(numParticles, dtype=np.int32)

        # Pick random thresholds in the cumulative probability's range [0,1]:
        some_threshes = np.random.rand(numParticles)


        # For each random threshold, find which sample in the ordered set is
        # the first one to push the cumulative probability above that
        # threshold, e.g. if the cumulative histogram goes from 0.23 to 0.26
        # between the 17th and 18th samples in the old distribution, and the
        # threshold is 0.234, then we'll want to propagate the 18th sample's w
        # (i.e. particle #18).

        for sampNum in range(numParticles): 
            thresh = some_threshes[sampNum]
            for index in range (numParticles):
                if cum_hist_of_weights[index] > thresh:
                    break
            samples_to_propagate[sampNum] = index

        # Note: it's ok if some of the old particles get picked repeatedly, while
        # others don't get picked at all.


        # =================================================
        # Visualize if you want
        # =================================================
#         plt.title('Cumulative histogram of probabilities for sorted list of particles')
#         plt.plot(np.zeros(numParticles), some_threshes,'b.')
#         plt.plot(range(0, numParticles), cum_hist_of_weights, 'rx-')
#         which_sample_ids = np.unique(samples_to_propagate)
#         how_many_of_each = np.bincount(np.ravel(samples_to_propagate))
#         for k in range(len(which_sample_ids)):
#            plt.plot(which_sample_ids[k], 0, 'bo-', markersize = 3 * how_many_of_each[k], markerfacecolor='white')
#         plt.xlabel('Indeces of all available samples, with larger blue circles for frequently re-sampled particles\n(Iteration %01d)' % iTime)
#         plt.ylabel('Cumulative probability');
#         plt.show()
        # =================================================
        # =================================================

        # Predict where the particles we sampled from the old distribution of 
        # state-space will go in the next time-step. This means we have to apply 
        # the motion model to each old sample.
        particles_new = np.zeros_like(particles_old)
        for particleNum in range(numParticles):
            # TO DO: Incorporate some noise, e.g. Gaussian noise with std 20,
            # into the current location (particles_old), to give a Brownian
            # motion model.
            particles_new[particleNum, :] = particles_old[samples_to_propagate[particleNum], :] + np.random.normal(0,std,numDims_w)
            #replace this
            
        # TO DO: Not initially, but change the motion model above to have
        # different degrees of freedom, and optionally completely different
        # motion models. See Extra Credit for more instructions.

        #calculate likelihood function
        likelihood = computeLikelihood(images[iTime], template)

        # plot results
        f, axarr = plt.subplots(1, 2)
        axarr[0].imshow(images[iTime])
        axarr[0].set_title('Particles')
        # now draw the particles onto the image
        axarr[0].plot(particles_new[:,1]+template.shape[1]/2, particles_new[:,0]+template.shape[0]/2, 'rx')

        #plot the likelihood
        axarr[1].imshow(likelihood)
        axarr[1].set_title('Likelihood')

        # From here we incorporate the data for the new state (time t):
        # The new particles accompanying predicted locations in state-space
        # for time t, are missing their weights: how well does each particle
        # explain the observations x_t?
        for particleNum in range(numParticles):

            # Convert the particle from state-space w to measurement-space x:
            # Note: that step is trivial here since both are in 2D space of image
            # coordinates

            # Within the loop, we evaluate the likelihood of each particle:
            particle = particles_new[particleNum, :]
            # Check that the predicted location is a place we can really evaluate
            # the likelihood.
            inFrame = particle[0] >= up_bound[iTime] and  particle[0] <= low_bound[iTime] and particle[1] >= left_bound[iTime] and particle[1] <= right_bound[iTime]            
            if inFrame:
                minX = particle[1]
                minY = particle[0]

                weight_of_samples[particleNum] = likelihood[int(minY), int(minX)]

            else:
                weight_of_samples[particleNum] = 0.0

        # TO DO: normalize the weights [done]
        weight_of_samples = weight_of_samples / np.sum(weight_of_samples) # replace this
        
        # find the location of the particle with highest weight
        indices = np.argsort(weight_of_samples,axis = 0)
        bestScoringParticles = particles_new[np.squeeze(indices[-15:]), :]
        plt.plot(bestScoringParticles[:,1], bestScoringParticles[:,0], 'rx')
        # Return the MAP of middle position. Add template.shape/2 because matchTemplate finds the position of the upper left corner 
        # of the template. We want to plot the centre of the template. 
        r[iTime,:] = bestScoringParticles[-1,1]+template.shape[1]/2,bestScoringParticles[-1,0]+template.shape[0]/2
        print(r[iTime,:])   
        plt.show()

        #print the original image and the position of the tracked corner.
        plt.imshow(images[iTime])
        plt.plot(r[iTime,0],r[iTime,1],'rx')
        plt.show()
        # Now we're done updating the state for time t. 
        # For Condensation, just clean up and prepare for the next round of 
        # predictions and measurements:
        particles_old = particles_new

    return r